In [26]:
import os 
import pandas as pd
from pathlib import Path
from tqdm.notebook import tqdm

sectors = ['agriculture', 'apparel', 'chemicals', 'machinery', 'materials', 
           'metals', 'miscellaneous', 'transport', 'minerals']

new_sectors = ['Agriculture', 'Apparel', 'Chemicals', 'Machinery', 'Materials', 
           'Metals', 'Miscellaneous', 'Transport', 'Minerals']
rename_columns = {s: ns for s, ns in zip(sectors, new_sectors)}
sectors = new_sectors

region2agg_dict = {
    'east africa': 'Africa', 
    'central africa': 'Africa',
    'north africa': 'Africa',
    'southern africa': 'Africa',
    'west africa': 'Africa',
    'caribbean': 'America',
    'central america': 'America',
    'south america': 'America',
    'north america': 'America',
    'central asia': 'Asia',
    'east asia': 'Asia',
    'south asia': 'Asia',
    'southeast asia': 'Asia',
    'west asia': 'Asia',
    'eastern europe': 'Europe', 
    'nordic': 'Europe',
    'southern europe': 'Europe',
    'western europe': 'Europe',
    'australia and new zealand': 'Oceania',
    'melanesia': 'Oceania',
    'micronesia': 'Oceania',
    'polynesia': 'Oceania',
}


project_dir = Path('C:\shares_TARIFF_project\comtrade_new\DID_country')


volume_df = pd.read_csv(project_dir / '2015_2021.csv', header=0)
volume_df = volume_df.rename(columns=rename_columns)
volume_df.head()

,source,year,month,Agriculture,Apparel,Chemicals,Machinery,Materials,Metals,Minerals,Miscellaneous,Transport
0,Afghanistan,2015,1,807693.0,508764.0,NaN,45445.0,NaN,4956.0,NaN,269303.0,20858.0
1,Afghanistan,2015,2,564296.0,170889.0,NaN,47867.0,NaN,4838991.0,NaN,403538.0,NaN
2,Afghanistan,2015,3,153039.0,731135.0,NaN,5582.0,NaN,146989.0,NaN,279085.0,NaN
3,Afghanistan,2015,4,384263.0,281741.0,NaN,26328.0,2856.0,474628.0,NaN,1070675.0,4451.0
4,Afghanistan,2015,5,57945.0,240599.0,NaN,10655.0,24119.0,1664127.0,NaN,676760.0,3607.0


In [27]:
# find the top 20 countries
volume_df['sum'] = volume_df.apply(lambda x: sum([x[i] for i in sectors]), axis=1)
top_countries = volume_df.groupby(by=['source'])[['sum']] \
                .sum().sort_values(by = ['sum'], ascending = False) \
                .head(22).index.tolist()

#print(volume_df)
volume_df = volume_df.query('source in @top_countries')


# 更新下国家或地区的名称，去掉空格（空格在did运算中会出路径问题）
volume_df['source'] = volume_df['source'].apply(lambda x: str(x).replace(' ', '_'))
top_countries = volume_df['source'].unique()
print(top_countries)

['Brazil' 'Canada' 'China' 'France' 'Germany' 'India' 'Indonesia'
 'Ireland' 'Israel' 'Italy' 'Japan' 'Malaysia' 'Mexico' 'Netherlands'
 'Other_Asia,_nes' 'Rep._of_Korea' 'Russian_Federation' 'Singapore'
 'Switzerland' 'Thailand' 'United_Kingdom' 'Viet_Nam']


In [28]:
region_df = pd.read_csv(project_dir / 'countries2regions_new.csv', header=0)
country2region_dict = {c:r for c, r in zip(region_df['source_country'], region_df['source_region'])}

region_df['agg'] = region_df['source_region'].map(region2agg_dict)

region2countries_dict = {}
for region, group_df in region_df.groupby(by=['agg']):
    region2countries_dict[region] = group_df['source_country'].tolist()

# 区分 USA和 Rest of America(ROA)

China_list = ['China']
region2countries_dict['China'] = China_list
roa_list = []
for c in region2countries_dict.get('Asia'):
    if c not in China_list:
        roa_list.append(c)
region2countries_dict['ROA'] = roa_list
del region2countries_dict['Asia']


C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_3336\2890156325.py:7: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for region, group_df in region_df.groupby(by=['agg']):


# get panel data - trade war

In [29]:
country2region_dict = {}
for rg, ct_list in region2countries_dict.items():
    for ct in ct_list:
        country2region_dict[ct] = rg



volume_sum_df = volume_df.groupby(by=['source', 'year', 'month']).sum()
volume_sum_df['region'], volume_sum_df['year'], volume_sum_df['month'] = zip(*volume_sum_df.index)
volume_sum_df.reset_index(drop=True, inplace=True)

# volume_mean_df = volume_sum_df.groupby(by=['year', 'month']).mean()
# volume_mean_df['year'], volume_mean_df['month'] = zip(*volume_mean_df.index)
# volume_mean_df.reset_index(drop=True, inplace=True)



# volume => percentage 转化为占比
years = volume_sum_df['year'].unique()
months = volume_sum_df['month'].unique()
for y in years:
    for m in months:
        query_df = volume_sum_df.query('year == @y and month == @m')[sectors]
        #print(query_df)
        _sum = query_df.sum()
        secter2sum_dict = {s:v for s, v in zip(_sum.index, _sum.tolist())}
        for idx in query_df.index:
            for s, v in secter2sum_dict.items():
                _v = volume_sum_df.loc[idx, s]
                p = _v / v * 100
                volume_sum_df.loc[idx, s] = p

print(volume_sum_df)

      Agriculture    Apparel  Chemicals  Machinery  Materials    Metals  \
0        4.260698   0.449057   0.975222   0.402846   2.959273  4.610218   
1        4.212676   0.396849   1.083890   0.349177   2.800043  3.163597   
2        4.211868   0.353062   0.720292   0.330025   2.819042  2.520451   
3        3.811707   0.542173   0.834325   0.378019   3.039443  2.548202   
4        3.735771   0.434122   0.792456   0.355908   3.262317  2.435849   
...           ...        ...        ...        ...        ...       ...   
1830     4.629573  21.344427   0.388025   5.234711   3.430279  1.663261   
1831     3.857840  16.901892   0.363505   5.145161   3.246773  1.759954   
1832     3.330712  16.311611   0.426333   6.408609   3.102815  1.842076   
1833     2.767803  14.883711   0.381127   6.995031   3.014355  2.217961   
1834     3.957236  17.472758   0.378028   6.319674   3.502277  2.408479   

      Minerals  Miscellaneous  Transport           sum    region  year  month  
0     3.451515     

In [30]:
# empty dir
panel_dir = project_dir / 'panels-trade-war'
for fp in panel_dir.rglob('*.csv'):
    os.remove(fp)

In [31]:
exp_years = [2018,2019]
com_years = [2015,2016,2017]



def get_did_panel(volume_sum_df, exp_years, com_years, sector, region):
    # volume_sum_df 中除去region外的求平均
    volume_mean_df = volume_sum_df.query('region != @region').groupby(by=['year', 'month']).mean()
    volume_mean_df['year'], volume_mean_df['month'] = zip(*volume_mean_df.index)
    volume_mean_df.reset_index(drop=True, inplace=True)
    com_df = volume_mean_df.query('year < 2020')[['year', 'month', sector]]
    com_df['name'] = 'mean'
    com_df.loc[com_df.query('year in @com_years').index, 'time'] = 0
    com_df.loc[com_df.query('year in @exp_years').index, 'time'] = 1
    com_df['time'] = com_df['time'].astype('int')
    com_df.loc[:, 'group'] = 0
    print(volume_mean_df)


    exp_df = volume_sum_df.query('year < 2020 and region == @region')[['region', 'year', 'month', sector]]
    exp_df = exp_df.rename(columns={'region': 'name'})
    exp_df.loc[exp_df.query('year in @com_years').index, 'time'] = 0
    exp_df.loc[exp_df.query('year in @exp_years').index, 'time'] = 1
    exp_df['time'] = exp_df['time'].astype('int')
    exp_df.loc[:, 'group'] = 1


    concat_df = pd.concat([exp_df, com_df])
    concat_df['did'] = concat_df.apply(lambda x: x['time'] * x['group'], axis=1)
    concat_df['date'] = concat_df.apply(lambda x: '{}-{}'.format(x['year'], x['month']), axis=1)
    del concat_df['year'], concat_df['month']
    concat_df = concat_df.rename(columns={sector: 'value'})
    # concat_df['value'] = np.log(concat_df['value'] + 1)
    return concat_df

for region in top_countries:
    for sector in sectors:
        panel_df = get_did_panel(volume_sum_df, exp_years, com_years, sector, region)
        panel_df.to_csv(panel_dir / f'{region}-{sector}.csv', header=True, index=False)

C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_3336\965263728.py:8: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  volume_mean_df = volume_sum_df.query('region != @region').groupby(by=['year', 'month']).mean()
C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_3336\965263728.py:8: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  volume_mean_df = volume_sum_df.query('region != @region').groupby(by=['year', 'month']).mean()
C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_3336\965263728.py:8: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only wi

    Agriculture   Apparel  Chemicals  Machinery  Materials    Metals  \
0      4.559014  4.740521   4.715466   4.742722   4.620987  4.542371   
1      4.561301  4.743007   4.710291   4.745277   4.628569  4.611257   
2      4.561340  4.745092   4.727605   4.746189   4.627665  4.641883   
3      5.062542  5.234622   5.219246   5.243262   5.103187  5.129042   
4      4.584011  4.741232   4.724169   4.744957   4.606556  4.645912   
..          ...       ...        ...        ...        ...       ...   
79     4.600226  4.743070   4.729583   4.743963   4.596065  4.614879   
80     4.581353  4.745898   4.726317   4.740987   4.592440  4.598820   
81     4.557084  4.746480   4.723012   4.740128   4.600889  4.656413   
82     4.557989  4.741563   4.718304   4.744750   4.590495  4.673229   
83     4.549179  4.741892   4.712522   4.741257   4.591383  4.650291   

    Minerals  Miscellaneous  Transport           sum  year  month  
0   4.597547       4.667270   4.721926  7.362274e+09  2015      1  

C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_3336\965263728.py:8: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  volume_mean_df = volume_sum_df.query('region != @region').groupby(by=['year', 'month']).mean()
C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_3336\965263728.py:8: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  volume_mean_df = volume_sum_df.query('region != @region').groupby(by=['year', 'month']).mean()
C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_3336\965263728.py:8: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only wi

    Agriculture   Apparel  Chemicals  Machinery  Materials    Metals  \
0      3.660803  4.680379   4.148595   4.546881   3.488935  3.946910   
1      3.585739  4.662675   4.180786   4.517309   3.399899  3.875454   
2      3.636015  4.672263   4.168223   4.543787   3.541681  3.928924   
3      3.936417  5.143235   4.711509   5.023590   3.833489  4.220738   
4      3.663419  4.662248   4.193140   4.556486   3.574887  3.952095   
..          ...       ...        ...        ...        ...       ...   
79     3.509258  4.701255   4.373152   4.577587   3.631457  3.815235   
80     3.527123  4.696685   4.378062   4.580126   3.637227  3.811812   
81     3.573620  4.698622   4.326327   4.581423   3.597362  3.751924   
82     3.640962  4.688724   4.336359   4.584107   3.594375  3.862138   
83     3.770605  4.698981   4.332081   4.586344   3.675591  3.878917   

    Minerals  Miscellaneous  Transport           sum  year  month  
0   1.716544       4.203283   3.696468  6.248113e+09  2015      1  

C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_3336\965263728.py:8: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  volume_mean_df = volume_sum_df.query('region != @region').groupby(by=['year', 'month']).mean()
C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_3336\965263728.py:8: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  volume_mean_df = volume_sum_df.query('region != @region').groupby(by=['year', 'month']).mean()
C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_3336\965263728.py:8: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only wi

...

    Agriculture   Apparel  Chemicals  Machinery  Materials    Metals  \
0      4.581122  4.700719   4.753109   4.655098   4.654578  4.678904   
1      4.617207  4.709814   4.751937   4.648370   4.670942  4.691201   
2      4.580032  4.695716   4.752651   4.647270   4.658520  4.673370   
3      5.025127  5.173857   5.249956   5.134133   5.124239  5.157432   
4      4.585863  4.705976   4.752936   4.652146   4.659593  4.668939   
..          ...       ...        ...        ...        ...       ...   
79     4.594535  4.705467   4.747829   4.609773   4.581439  4.658835   
80     4.608569  4.709146   4.747847   4.611124   4.570722  4.652531   
81     4.611241  4.704907   4.745391   4.624228   4.580028  4.656523   
82     4.612443  4.705601   4.744772   4.621634   4.575368  4.656181   
83     4.589788  4.703506   4.744065   4.615069   4.584482  4.658531   

    Minerals  Miscellaneous  Transport           sum  year  month  
0   4.761867       4.730842   4.746729  7.374752e+09  2015      1  

C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_3336\965263728.py:8: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  volume_mean_df = volume_sum_df.query('region != @region').groupby(by=['year', 'month']).mean()
C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_3336\965263728.py:8: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  volume_mean_df = volume_sum_df.query('region != @region').groupby(by=['year', 'month']).mean()
C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_3336\965263728.py:8: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only wi

    Agriculture   Apparel  Chemicals  Machinery  Materials    Metals  \
0      4.630619  4.745769   4.549285   4.658780   4.689287  4.643656   
1      4.651440  4.740380   4.421769   4.641129   4.669188  4.663122   
2      4.647843  4.743687   4.495871   4.657744   4.678365  4.674573   
3      5.116284  5.239183   4.659666   5.145449   5.170355  5.159653   
4      4.622951  4.741728   4.246823   4.655718   4.684121  4.681029   
..          ...       ...        ...        ...        ...       ...   
79     4.682513  4.747492   4.566131   4.682524   4.703390  4.672411   
80     4.662137  4.743701   4.558198   4.682443   4.700419  4.670755   
81     4.671946  4.745250   4.615040   4.682007   4.705886  4.703434   
82     4.666368  4.740340   4.551268   4.685702   4.696996  4.653113   
83     4.684797  4.741729   4.550050   4.689797   4.699553  4.675884   

    Minerals  Miscellaneous  Transport           sum  year  month  
0   4.613825       4.489088   4.628736  7.281284e+09  2015      1  

    Agriculture   Apparel  Chemicals  Machinery  Materials    Metals  \
0      4.602587  4.174772   4.756380   4.698168   4.730539  4.736067   
1      4.639490  4.178541   4.757352   4.698049   4.730047  4.736725   
2      4.618873  4.125869   4.754631   4.687136   4.723476  4.739293   
3      5.079778  4.437907   5.257060   5.181953   5.217979  5.225495   
4      4.571403  4.136053   4.755316   4.680918   4.722794  4.736255   
..          ...       ...        ...        ...        ...       ...   
79     4.541449  3.745503   4.743427   4.512633   4.598558  4.682702   
80     4.578198  3.957053   4.744595   4.516897   4.607297  4.678097   
81     4.603299  3.985161   4.741603   4.456733   4.614152  4.674187   
82     4.630105  4.053157   4.743756   4.428808   4.618364  4.656288   
83     4.573465  3.929869   4.743903   4.460968   4.595130  4.647215   

    Minerals  Miscellaneous  Transport           sum  year  month  
0   4.761472       4.628010   4.756778  7.349464e+09  2015      1  

C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_3336\965263728.py:8: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  volume_mean_df = volume_sum_df.query('region != @region').groupby(by=['year', 'month']).mean()
C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_3336\965263728.py:8: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  volume_mean_df = volume_sum_df.query('region != @region').groupby(by=['year', 'month']).mean()
C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_3336\965263728.py:8: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only wi

# DID-trade-war

In [32]:
import re
import ipystata

region_list = list(top_countries)
sector_list = sectors

# empty dir
panel_dir = project_dir / 'temp'
for fp in panel_dir.rglob('*.txt'):
    os.remove(fp)

In [33]:
%%stata -i sector_list -i region_list
foreach region in `region_list' {
    display "------------------------`region'---------------------------------"
    clear
    cd "C:\shares_TARIFF_project\comtrade_new\DID_country"
    foreach sector in `sector_list' {
        import delimited "./panels-trade-war/`region'-`sector'.csv"
        egen time1 = group(date)
        xtset time1 group
        reghdfe value did, absorb(time1 group)
        outreg2 using "./temp/tradewar-`region'-`sector'.txt"
        clear
    }
}


------------------------Brazil---------------------------------
C:\shares_TARIFF_project\comtrade_new\DID_country
(6 vars, 120 obs)
       panel variable:  time1 (strongly balanced)
        time variable:  group, 0 to 1
                delta:  1 unit
(MWFE estimator converged in 2 iterations)

HDFE Linear regression                            Number of obs   =        120
Absorbing 2 HDFE groups                           F(   1,     58) =       5.06
                                                  Prob > F        =     0.0283
                                                  R-squared       =     0.7174
                                                  Adj R-squared   =     0.4203
                                                  Within R-sq.    =     0.0802
                                                  Root MSE        =     0.4603

------------------------------------------------------------------------------
       value |      Coef.   Std. Err.      t    P>|t|     [95% Conf. In

In [34]:
def get_result(result_fp):
    with open(result_fp, 'r') as fp:
        result_str = fp.read()
        match_str = re.findall(r'did\t(.*?)\n', result_str)[0]
        star_count = match_str.count('*')
        match_str = match_str.replace(',', '')
        if match_str == '-':# 若匹配到空，判定不显著
            significant = -1
            coff = -999
        else:
            if star_count == 0:
                significant = -1
                coff = float(match_str)
            else:
                coff = float(match_str.split('*')[0])
                if star_count == 1:
                    significant = 0.1
                elif star_count == 2:
                    significant = 0.05
                else:
                    significant = 0.01
    return coff, significant

result_list = []
for region in region_list:
    for sector in sector_list:
        result_fp = project_dir / 'temp' / f'tradewar-{region}-{sector}.txt'
        coff, significant = get_result(result_fp)
        result_list.append((region, sector, coff, significant))
result_df = pd.DataFrame(result_list, columns=['region', 'sector', 'coff', 'significance'])



# # 将coff的值区间缩放至 -1,1
# data = result_df['coff']
# data_t = (data-data.mean())/(data.max()-data.min())
# result_df['coff'] = data_t
result_df.to_csv('C:\shares_TARIFF_project\comtrade_new\DID_country\did-result-trade-war-meso.csv', header=True, index=False)

# get panel data - pandemic

In [35]:
country2region_dict = {}
for rg, ct_list in region2countries_dict.items():
    for ct in ct_list:
        country2region_dict[ct] = rg



volume_sum_df = volume_df.groupby(by=['source', 'year', 'month']).sum()
volume_sum_df['region'], volume_sum_df['year'], volume_sum_df['month'] = zip(*volume_sum_df.index)
volume_sum_df.reset_index(drop=True, inplace=True)

# volume_mean_df = volume_sum_df.groupby(by=['year', 'month']).mean()
# volume_mean_df['year'], volume_mean_df['month'] = zip(*volume_mean_df.index)
# volume_mean_df.reset_index(drop=True, inplace=True)



# volume => percentage 转化为占比
years = volume_sum_df['year'].unique()
months = volume_sum_df['month'].unique()
for y in years:
    for m in months:
        query_df = volume_sum_df.query('year == @y and month == @m')[sectors]
        _sum = query_df.sum()
        secter2sum_dict = {s:v for s, v in zip(_sum.index, _sum.tolist())}
        for idx in query_df.index:
            for s, v in secter2sum_dict.items():
                _v = volume_sum_df.loc[idx, s]
                p = _v / v * 100
                volume_sum_df.loc[idx, s] = p


In [36]:
# empty dir
panel_dir = project_dir / 'panels-pandemic'
for fp in panel_dir.rglob('*.csv'):
    os.remove(fp)

In [37]:
exp_years = [2020,2021]
com_years = [2018,2019]


def get_did_panel(volume_sum_df, exp_years, com_years, sector, region):
    
    volume_mean_df = volume_sum_df.query('region != @region').groupby(by=['year', 'month']).mean()
    volume_mean_df['year'], volume_mean_df['month'] = zip(*volume_mean_df.index)
    volume_mean_df.reset_index(drop=True, inplace=True)
    
    com_df = volume_mean_df[['year', 'month', sector]]
    com_df['name'] = 'mean'
    com_df.loc[com_df.query('year in @com_years').index, 'time'] = 0
    com_df.loc[com_df.query('year in @exp_years').index, 'time'] = 1
    com_df = com_df.dropna()
    com_df['time'] = com_df['time'].astype('int')
    com_df.loc[:, 'group'] = 0


    exp_df = volume_sum_df.query('region == @region')[['region', 'year', 'month', sector]]
    exp_df = exp_df.rename(columns={'region': 'name'})
    exp_df.loc[exp_df.query('year in @com_years').index, 'time'] = 0
    exp_df.loc[exp_df.query('year in @exp_years').index, 'time'] = 1
    exp_df = exp_df.dropna()
    exp_df['time'] = exp_df['time'].astype('int')
    exp_df.loc[:, 'group'] = 1


    concat_df = pd.concat([exp_df, com_df])
    concat_df['did'] = concat_df.apply(lambda x: x['time'] * x['group'], axis=1)
    concat_df['date'] = concat_df.apply(lambda x: '{}-{}'.format(x['year'], x['month']), axis=1)
    del concat_df['year'], concat_df['month']
    concat_df = concat_df.rename(columns={sector: 'value'})
    # concat_df['value'] = np.log(concat_df['value'] + 1)
    return concat_df

for region in top_countries:
    for sector in sectors:
        panel_df = get_did_panel(volume_sum_df, exp_years, com_years, sector, region)
        panel_df.to_csv(panel_dir / f'{region}-{sector}.csv', header=True, index=False)

C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_3336\3514503499.py:7: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  volume_mean_df = volume_sum_df.query('region != @region').groupby(by=['year', 'month']).mean()
C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_3336\3514503499.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  com_df['name'] = 'mean'
C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_3336\3514503499.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value 

C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_3336\3514503499.py:7: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  volume_mean_df = volume_sum_df.query('region != @region').groupby(by=['year', 'month']).mean()
C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_3336\3514503499.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  com_df['name'] = 'mean'
C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_3336\3514503499.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value 

C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_3336\3514503499.py:7: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  volume_mean_df = volume_sum_df.query('region != @region').groupby(by=['year', 'month']).mean()
C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_3336\3514503499.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  com_df['name'] = 'mean'
C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_3336\3514503499.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value 

C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_3336\3514503499.py:7: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  volume_mean_df = volume_sum_df.query('region != @region').groupby(by=['year', 'month']).mean()
C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_3336\3514503499.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  com_df['name'] = 'mean'
C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_3336\3514503499.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value 

C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_3336\3514503499.py:7: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  volume_mean_df = volume_sum_df.query('region != @region').groupby(by=['year', 'month']).mean()
C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_3336\3514503499.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  com_df['name'] = 'mean'
C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_3336\3514503499.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value 

...

C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_3336\3514503499.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exp_df['time'] = exp_df['time'].astype('int')
C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_3336\3514503499.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exp_df.loc[:, 'group'] = 1
C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_3336\3514503499.py:7: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default

C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_3336\3514503499.py:7: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  volume_mean_df = volume_sum_df.query('region != @region').groupby(by=['year', 'month']).mean()
C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_3336\3514503499.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  com_df['name'] = 'mean'
C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_3336\3514503499.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value 

C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_3336\3514503499.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exp_df['time'] = exp_df['time'].astype('int')
C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_3336\3514503499.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exp_df.loc[:, 'group'] = 1
C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_3336\3514503499.py:7: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default

C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_3336\3514503499.py:7: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  volume_mean_df = volume_sum_df.query('region != @region').groupby(by=['year', 'month']).mean()
C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_3336\3514503499.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  com_df['name'] = 'mean'
C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_3336\3514503499.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value 

C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_3336\3514503499.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exp_df['time'] = exp_df['time'].astype('int')
C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_3336\3514503499.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exp_df.loc[:, 'group'] = 1
C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_3336\3514503499.py:7: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default

# did-pandemic

In [38]:
import re
import ipystata

region_list = list(top_countries)
sector_list = sectors


# empty dir
panel_dir = project_dir / 'temp-1'
for fp in panel_dir.rglob('*.txt'):
    os.remove(fp)

In [39]:
%%stata -i sector_list -i region_list
foreach region in `region_list' {
    display "------------------------`region'---------------------------------"
    clear
    cd "C:\shares_TARIFF_project\comtrade_new\DID_country"
    foreach sector in `sector_list' {
        import delimited "./panels-pandemic/`region'-`sector'.csv"
        egen time1 = group(date)
        xtset time1 group
        reghdfe value did, absorb(time1 group)
        outreg2 using "./temp-1/pandemic-`region'-`sector'.txt"
        clear
    }
}


------------------------Brazil---------------------------------
C:\shares_TARIFF_project\comtrade_new\DID_country
(6 vars, 96 obs)
       panel variable:  time1 (strongly balanced)
        time variable:  group, 0 to 1
                delta:  1 unit
(MWFE estimator converged in 2 iterations)

HDFE Linear regression                            Number of obs   =         96
Absorbing 2 HDFE groups                           F(   1,     46) =       0.30
                                                  Prob > F        =     0.5871
                                                  R-squared       =     0.7722
                                                  Adj R-squared   =     0.5296
                                                  Within R-sq.    =     0.0065
                                                  Root MSE        =     0.4831

------------------------------------------------------------------------------
       value |      Coef.   Std. Err.      t    P>|t|     [95% Conf. Int

In [40]:
def get_result(result_fp):
    with open(result_fp, 'r') as fp:
        result_str = fp.read()
        match_str = re.findall(r'did\t(.*?)\n', result_str)[0]
        star_count = match_str.count('*')
        match_str = match_str.replace(',', '')
        if match_str == '-':# 若匹配到空，判定不显著
            significant = -1
            coff = -999
        else:
            if star_count == 0:
                significant = -1
                coff = float(match_str)
            else:
                coff = float(match_str.split('*')[0])
                if star_count == 1:
                    significant = 0.1
                elif star_count == 2:
                    significant = 0.05
                else:
                    significant = 0.01
    return coff, significant

result_list = []
for region in region_list:
    for sector in sector_list:
        result_fp = project_dir / 'temp-1' / f'pandemic-{region}-{sector}.txt'
        coff, significant = get_result(result_fp)
        result_list.append((region, sector, coff, significant))
result_df = pd.DataFrame(result_list, columns=['region', 'sector', 'coff', 'significance'])


# # 将coff的值区间缩放至 -1,1
# data = result_df['coff']
# data_t = (data-data.mean())/(data.max()-data.min())
# result_df['coff'] = data_t
result_df.to_csv('C:\shares_TARIFF_project\comtrade_new\DID_country\did-result-pandemic-meso-true.csv', header=True, index=False)
#result_df.to_csv('C:\\shares_TARIFF_project\\comtrade_new\\DID_country_cancel2018\\did-summary\\did-result-pandemic-meso-true.csv', header=True, index=False)
# result_df

# get panel data -two events

In [47]:
country2region_dict = {}
for rg, ct_list in region2countries_dict.items():
    for ct in ct_list:
        country2region_dict[ct] = rg



volume_sum_df = volume_df.groupby(by=['source', 'year', 'month']).sum()
volume_sum_df['region'], volume_sum_df['year'], volume_sum_df['month'] = zip(*volume_sum_df.index)
volume_sum_df.reset_index(drop=True, inplace=True)

# volume_mean_df = volume_sum_df.groupby(by=['year', 'month']).mean()
# volume_mean_df['year'], volume_mean_df['month'] = zip(*volume_mean_df.index)
# volume_mean_df.reset_index(drop=True, inplace=True)



# volume => percentage 转化为占比
years = volume_sum_df['year'].unique()
months = volume_sum_df['month'].unique()
for y in years:
    for m in months:
        query_df = volume_sum_df.query('year == @y and month == @m')[sectors]
        _sum = query_df.sum()
        secter2sum_dict = {s:v for s, v in zip(_sum.index, _sum.tolist())}
        for idx in query_df.index:
            for s, v in secter2sum_dict.items():
                _v = volume_sum_df.loc[idx, s]
                p = _v / v * 100
                volume_sum_df.loc[idx, s] = p

C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_3336\265871262.py:8: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  volume_sum_df = volume_df.groupby(by=['source', 'year', 'month']).sum()


In [48]:
# empty dir
panel_dir = project_dir / 'panels-two-events'
for fp in panel_dir.rglob('*.csv'):
    os.remove(fp)

In [49]:
exp_years = [2020,2021]
com_years = [2015,2016,2017]


def get_did_panel(volume_sum_df, exp_years, com_years, sector, region):
    
    volume_mean_df = volume_sum_df.query('region != @region').groupby(by=['year', 'month']).mean()
    volume_mean_df['year'], volume_mean_df['month'] = zip(*volume_mean_df.index)
    volume_mean_df.reset_index(drop=True, inplace=True)
    
    com_df = volume_mean_df[['year', 'month', sector]]
    com_df['name'] = 'mean'
    com_df.loc[com_df.query('year in @com_years').index, 'time'] = 0
    com_df.loc[com_df.query('year in @exp_years').index, 'time'] = 1
    com_df = com_df.dropna()
    com_df['time'] = com_df['time'].astype('int')
    com_df.loc[:, 'group'] = 0


    exp_df = volume_sum_df.query('region == @region')[['region', 'year', 'month', sector]]
    exp_df = exp_df.rename(columns={'region': 'name'})
    exp_df.loc[exp_df.query('year in @com_years').index, 'time'] = 0
    exp_df.loc[exp_df.query('year in @exp_years').index, 'time'] = 1
    exp_df = exp_df.dropna()
    exp_df['time'] = exp_df['time'].astype('int')
    exp_df.loc[:, 'group'] = 1


    concat_df = pd.concat([exp_df, com_df])
    concat_df['did'] = concat_df.apply(lambda x: x['time'] * x['group'], axis=1)
    concat_df['date'] = concat_df.apply(lambda x: '{}-{}'.format(x['year'], x['month']), axis=1)
    del concat_df['year'], concat_df['month']
    concat_df = concat_df.rename(columns={sector: 'value'})
    # concat_df['value'] = np.log(concat_df['value'] + 1)
    return concat_df

for region in top_countries:
    for sector in sectors:
        panel_df = get_did_panel(volume_sum_df, exp_years, com_years, sector, region)
        panel_df.to_csv(panel_dir / f'{region}-{sector}.csv', header=True, index=False)

C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_3336\2979130469.py:7: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  volume_mean_df = volume_sum_df.query('region != @region').groupby(by=['year', 'month']).mean()
C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_3336\2979130469.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  com_df['name'] = 'mean'
C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_3336\2979130469.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value 

C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_3336\2979130469.py:7: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  volume_mean_df = volume_sum_df.query('region != @region').groupby(by=['year', 'month']).mean()
C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_3336\2979130469.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  com_df['name'] = 'mean'
C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_3336\2979130469.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value 

C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_3336\2979130469.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exp_df['time'] = exp_df['time'].astype('int')
C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_3336\2979130469.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exp_df.loc[:, 'group'] = 1
C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_3336\2979130469.py:7: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default

C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_3336\2979130469.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exp_df['time'] = exp_df['time'].astype('int')
C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_3336\2979130469.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exp_df.loc[:, 'group'] = 1
C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_3336\2979130469.py:7: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default

C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_3336\2979130469.py:7: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  volume_mean_df = volume_sum_df.query('region != @region').groupby(by=['year', 'month']).mean()
C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_3336\2979130469.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  com_df['name'] = 'mean'
C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_3336\2979130469.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value 

...

C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_3336\2979130469.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exp_df['time'] = exp_df['time'].astype('int')
C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_3336\2979130469.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exp_df.loc[:, 'group'] = 1
C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_3336\2979130469.py:7: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default

C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_3336\2979130469.py:7: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  volume_mean_df = volume_sum_df.query('region != @region').groupby(by=['year', 'month']).mean()
C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_3336\2979130469.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  com_df['name'] = 'mean'
C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_3336\2979130469.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value 

C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_3336\2979130469.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exp_df['time'] = exp_df['time'].astype('int')
C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_3336\2979130469.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exp_df.loc[:, 'group'] = 1
C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_3336\2979130469.py:7: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default

C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_3336\2979130469.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exp_df['time'] = exp_df['time'].astype('int')
C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_3336\2979130469.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exp_df.loc[:, 'group'] = 1
C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_3336\2979130469.py:7: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default

C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_3336\2979130469.py:7: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  volume_mean_df = volume_sum_df.query('region != @region').groupby(by=['year', 'month']).mean()
C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_3336\2979130469.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  com_df['name'] = 'mean'
C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_3336\2979130469.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value 

# DID-two-events

In [51]:
import re
import ipystata

region_list = list(top_countries)
sector_list = sectors


# empty dir
panel_dir = project_dir / 'temp-2'
for fp in panel_dir.rglob('*.txt'):
    os.remove(fp)

In [52]:
%%stata -i sector_list -i region_list
foreach region in `region_list' {
    display "------------------------`region'---------------------------------"
    clear
    cd "C:\shares_TARIFF_project\comtrade_new\DID_country"
    foreach sector in `sector_list' {
        import delimited "./panels-two-events/`region'-`sector'.csv"
        egen time1 = group(date)
        xtset time1 group
        reghdfe value did, absorb(time1 group)
        outreg2 using "./temp-2/two-events-`region'-`sector'.txt"
        clear
    }
}


------------------------Brazil---------------------------------
C:\shares_TARIFF_project\comtrade_new\DID_country
(6 vars, 120 obs)
       panel variable:  time1 (strongly balanced)
        time variable:  group, 0 to 1
                delta:  1 unit
(MWFE estimator converged in 2 iterations)

HDFE Linear regression                            Number of obs   =        120
Absorbing 2 HDFE groups                           F(   1,     58) =       8.17
                                                  Prob > F        =     0.0059
                                                  R-squared       =     0.7378
                                                  Adj R-squared   =     0.4621
                                                  Within R-sq.    =     0.1235
                                                  Root MSE        =     0.4633

------------------------------------------------------------------------------
       value |      Coef.   Std. Err.      t    P>|t|     [95% Conf. In

In [54]:
def get_result(result_fp):
    with open(result_fp, 'r') as fp:
        result_str = fp.read()
        match_str = re.findall(r'did\t(.*?)\n', result_str)[0]
        star_count = match_str.count('*')
        match_str = match_str.replace(',', '')
        if match_str == '-':# 若匹配到空，判定不显著
            significant = -1
            coff = -999
        else:
            if star_count == 0:
                significant = -1
                coff = float(match_str)
            else:
                coff = float(match_str.split('*')[0])
                if star_count == 1:
                    significant = 0.1
                elif star_count == 2:
                    significant = 0.05
                else:
                    significant = 0.01
    return coff, significant

result_list = []
for region in region_list:
    for sector in sector_list:
        result_fp = project_dir / 'temp-2' / f'two-events-{region}-{sector}.txt'
        coff, significant = get_result(result_fp)
        result_list.append((region, sector, coff, significant))
result_df = pd.DataFrame(result_list, columns=['region', 'sector', 'coff', 'significance'])


# # 将coff的值区间缩放至 -1,1
# data = result_df['coff']
# data_t = (data-data.mean())/(data.max()-data.min())
# result_df['coff'] = data_t
result_df.to_csv('C:\shares_TARIFF_project\comtrade_new\DID_country\did-result-two-events.csv', header=True, index=False)
#result_df.to_csv('C:\\shares_TARIFF_project\\comtrade_new\\DID_country_cancel2018\\did-summary\\did-result-pandemic-meso-true.csv', header=True, index=False)
# result_df

In [56]:
result_two_events_fp = 'C:\shares_TARIFF_project\comtrade_new\DID_country\did-result-two-events.csv'
result_two_events_df = pd.read_csv(result_two_events_fp)
result_pandemic_fp = 'C:\shares_TARIFF_project\comtrade_new\DID_country\did-result-pandemic-meso-true.csv'
result_pandemic_df = pd.read_csv(result_pandemic_fp)
result_trade_war_fp = 'C:\shares_TARIFF_project\comtrade_new\DID_country\did-result-trade-war-meso.csv'
result_trade_war_df = pd.read_csv(result_trade_war_fp)

import numpy as np

# 如果sign=-1（不显著），设置coff为NaN
def func(x):
    if x['significance'] == -1:
        return np.nan
    else:
        return x['coff']
result_two_events_df['coff'] = result_two_events_df.apply(func, axis=1)
result_pandemic_df['coff'] = result_pandemic_df.apply(func, axis=1)
result_trade_war_df['coff'] = result_trade_war_df.apply(func, axis=1)

merge_df = pd.merge(result_two_events_df,result_pandemic_df,  
         left_on=['region', 'sector'], 
         right_on=['region', 'sector'], 
         how='inner') \
    .rename(columns={'coff_x': 'coff_two_events', 
                     'coff_y': 'coff_pandemic',
                     'significance_x': 'sign_two_events', 
                     'significance_y': 'sign_pandemic'})


merge_df = pd.merge(merge_df, result_trade_war_df, 
         left_on=['region', 'sector'], 
         right_on=['region', 'sector'], 
         how='inner') \
    .rename(columns={'coff': 'coff_trade_war', 
                     'significance': 'sign_trade_war'})





# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()
# data = merge_df['coff_tradewar'].tolist()
# data_t = scaler.fit_transform(np.array(data).reshape(-1,1)).reshape(1,-1).tolist()[0]
# merge_df['coff_tradewar'] = data_t


# data = merge_df['coff_pandemic'].tolist()
# data_t = scaler.fit_transform(np.array(data).reshape(-1,1)).reshape(1,-1).tolist()[0]
# merge_df['coff_pandemic'] = data_t

merge_df.to_csv('C:\shares_TARIFF_project\comtrade_new\DID_country\did-result-merge-meso.csv', header=True, index=False)
merge_df.head()

,region,sector,coff_two_events,sign_two_events,coff_pandemic,sign_pandemic,coff_trade_war,sign_trade_war
0,Brazil,Agriculture,-0.4940,0.01,NaN,-1.00,-0.3860,0.05
1,Brazil,Apparel,-0.0529,0.10,NaN,-1.00,-0.0664,0.05
2,Brazil,Chemicals,-0.2160,0.01,-0.202,0.01,NaN,-1.00
3,Brazil,Machinery,0.0620,0.05,NaN,-1.00,NaN,-1.00
4,Brazil,Materials,NaN,-1.00,NaN,-1.00,NaN,-1.00


In [58]:
for coff_str in ['coff_two_events', 'coff_pandemic','coff_trade_war']:
    pivot_df = merge_df[['region', 'sector', coff_str]].set_index('region') \
        .pivot(columns='sector', values=coff_str)
    pivot_df.to_csv(f'C:\shares_TARIFF_project\comtrade_new\DID_country\pivot-{coff_str}-meso.csv', header=True, index=True)